# **Отчёт о выполнении работы по теме «Основы веб-скрапинга»**
![](https://drive.google.com/uc?export=view&id=1hWGH-1bhAzCdMJwQbuXbqC6vAwkbLHVM)
***

In [1]:
import requests
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
import re
from lxml import html
import time

%load_ext lab_black

In [2]:
#!pip install nb_black

# **Доработанное задание**

<div class="alert alert-block alert-info">

## **Обязательная часть**

Вам необходимо написать функцию, которая будет основана на поиске по сайту [habr.com](https://habr.com/ru/search/). Функция в качестве параметра должна принимать список запросов для поиска (например, ['python', 'анализ данных']) и на основе материалов, попавших в результаты поиска по каждому запросу, возвращать датафрейм вида:

><дата> - <заголовок> - <ссылка на материал>

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.
***

In [3]:
def search_habr_megapost(requests_list):

    """Функция осуществляет поиск по сайту habr.com на основании списка запросов и возвращает первую страницу результатов поиска по каждому слову из списка.
    Дубликаты найденных материалов удаляются"""

    df_result = pd.DataFrame()
    for search_query in requests_list:
        response = requests.get(
            "https://habr.com/ru/search/", params={"q": search_query}
        )
        soup = BeautifulSoup(response.text, "html")
        find_ = soup.find_all("article", class_="tm-articles-list__item")
        for el in find_:
            try:
                title = el.find(
                    "h2",
                    class_="tm-article-snippet__title tm-article-snippet__title_h2",
                ).text
                link = el.find("a", "tm-article-snippet__title-link").get("href")
                date = el.find(
                    "span", "tm-article-snippet__datetime-published"
                ).text.split(" в ")[0]
            except AttributeError:
                title = el.find(
                    "a", class_="tm-megapost-snippet__link tm-megapost-snippet__card"
                ).text
                link = el.find(
                    "a", "tm-megapost-snippet__link tm-megapost-snippet__card"
                ).get("href")
                date = el.find(
                    "time", "tm-megapost-snippet__datetime-published"
                ).text.split(" в ")[0]
            row = {"date": date, "title": title, "link": "https://habr.com" + link}
            df_result = pd.concat([df_result, pd.DataFrame([row])])
        df_result = df_result.drop_duplicates().reset_index(drop=True)
    return df_result

In [4]:
requests_list = ["python", "SDSJ", "head"]
search_habr_megapost(requests_list)

,date,title,link
0,20 января,Курс «Python для инженеров». Старт 3 потока 31 января,https://habr.com/ru/company/southbridge/news/t/646825/
1,13 декабря 2021,"Жаждущим автоматизации: открытый урок «ChatOps c Errbot на Python», 21 декабря",https://habr.com/ru/company/southbridge/news/t/595093/
2,21 апреля 2020,"Вышел Python 2.7.18, последний релиз ветки Python 2.x",https://habr.com/ru/news/t/498364/
3,6 июля 2021,Python Community Meetup 8/07: видео и материалы встречи,https://habr.com/ru/company/raiffeisenbank/news/t/566370/
4,13 января,"Открытый урок «Пишем Custom Prometheus Exporter на Python», 19 января",https://habr.com/ru/company/southbridge/news/t/645485/
...,...,...,...
40,17 сентября 2012,Паттерны проектирования (Head First Design Patterns),https://habr.com/ru/company/piter/blog/151626/
41,29 марта 2018,Книга «Head First. Паттерны проектирования. Обновленное юбилейное издание»,https://habr.com/ru/company/piter/blog/352202/
42,18 июня 2021,"Rust 1.53.0: IntoIterator для массивов, ""|"" в шаблонах, Unicode-идентификаторы, поддержка имени HEAD-ветки в Cargo",https://habr.com/ru/post/563548/
43,22 ноября 2007,Firefox и HTTP HEAD метод,https://habr.com/ru/post/16539/


In [5]:
#!pip install mitosheet

<div class="alert alert-block alert-info">

## **Дополнительная часть (необязательная)**

Функция из обязательной части задания должна быть расширена следующим образом:

- кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента 4 необходимо получить материалы с первых 4 страниц результатов;
- в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:
> <дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>

***

In [5]:
def search_habr_full(requests_list, page_count):

    """Функция осуществляет поиск по сайту habr.com на основании списка запросов и возвращает указанное число страниц"""

    df_result = pd.DataFrame()
    for search_query in requests_list:
        for page in range(1, page_count + 1):
            url = "https://habr.com/ru/search/page" + str(page) + "/"
            params = {"q": search_query}
            response = requests.get(url, params=params)
            time.sleep(0.33)
            soup = BeautifulSoup(response.text, "html")
            find_ = soup.find_all("article", class_="tm-articles-list__item")
            df_page_result = pd.DataFrame()
            for el in find_:
                # здесь название, ссылка и "дата" в виде текста, это работает
                try:
                    title = el.find(
                        "h2",
                        class_="tm-article-snippet__title tm-article-snippet__title_h2",
                    ).text
                    link = el.find("a", "tm-article-snippet__title-link").get("href")
                    date = el.find(
                        "span", "tm-article-snippet__datetime-published"
                    ).text.split(" в ")[0]
                except AttributeError:
                    title = el.find(
                        "a",
                        class_="tm-megapost-snippet__link tm-megapost-snippet__card",
                    ).text
                    link = el.find(
                        "a", "tm-megapost-snippet__link tm-megapost-snippet__card"
                    ).get("href")
                    date = el.find(
                        "time", "tm-megapost-snippet__datetime-published"
                    ).text.split(" в ")[0]
                # здесь лайки. class лайков зависит от качественной характеристики оценки, перебираем возмождные варианты
                for element_class in [
                    " ",
                    " tm-votes-meter__value_positive ",
                    " tm-votes-meter__value_negative ",
                ]:
                    class_ = (
                        "tm-votes-meter__value tm-votes-meter__value"
                        + element_class
                        + "tm-votes-meter__value_appearance-article tm-votes-meter__value_rating"
                    )
                    try:
                        likes = el.find("span", class_=class_).text
                        break
                    except AttributeError:
                        pass
                # здесь заканчиваем разбирать лайки и приступаем к поиску полного текста поста по ранее найденной ссылке
                req = requests.get("https://habr.com" + link).text
                soup_full_text = BeautifulSoup(req)
                # здесь перебираем номера версий для постов. Вообще-то версий, если проверять вручную, всего 2, но, для надёжности, сделаем больше
                version = 1
                while version <= 10:
                    try:
                        class_ = (
                            "article-formatted-body article-formatted-body article-formatted-body_version-"
                            + str(version)
                        )
                        full_text = soup_full_text.find(
                            "div", class_=class_
                        ).text.strip()
                        full_text = re.sub(r"\s+", " ", full_text)
                        break
                    except AttributeError:
                        version += 1
                        full_text = "megapost"
                # здесь неуклюже вытаскиваем полный текст из этиго вашего megapost'a
                if full_text == "megapost":
                    resp = requests.get("https://habr.com" + link)
                    soup = BeautifulSoup(resp.text, "html")
                    find_title_text = soup.find_all(
                        "div", class_="t119__preface t-descr t-opacity_70"
                    )
                    find_text = soup.find_all("div", class_="t-text t-text_md")
                    full_text = ""
                    try:
                        i = 0
                        while True:
                            full_text += find_title_text[i].text
                            i += 1
                    except IndexError:
                        pass
                    try:
                        j = 0
                        while True:
                            full_text += find_text[j].text
                            j += 1
                    except IndexError:
                        pass
                # разобрались со всеми данными, формируем страницу
                row = {
                    "date": date,
                    "title": title,
                    "link": "https://habr.com" + link,
                    "text": full_text,
                    "likes": likes,
                }
                df_page_result = pd.concat([df_page_result, pd.DataFrame([row])])
                df_page_result = df_page_result.drop_duplicates()
                # присоединяем страницу к ранее найденному
                df_result = pd.concat([df_result, df_page_result])
    # объединяем найденное и возаращаем результат поиска
    return df_result.drop_duplicates().reset_index(drop=True)

In [7]:
# SDSJ
search_habr_full(requests_list=["sql", "SDSJ"], page_count=1)

date                                              title  \
0    25  октября  2021  Ошибка в экспоненциальной форме записи чисел в...   
1          7  апреля    Яндекс Практикум запускает курс «SQL для работ...   
2       30  июня  2021  Решения Quest для управления и мониторинга Mic...   
3        24  августа    Python, SQL, С и Java стали самыми популярными...   
4        30  октября             Топ полезных SQL-запросов для PostgreSQL   
5    22  декабря  2020  DataLine запускает кластер для нагруженных баз...   
6    14  октября  2021          Бесплатный курс-симулятор: введение в SQL   
7           21  июня    SQL Camp — летняя образовательная программа Ро...   
8            5  июля    Разработчики Django исправили уязвимость, кото...   
9     5  октября  2021  Внимание! Приглашаем программистов 1С и SQL к ...   
10     7  апреля  2016  Разнообразие версий Microsoft SQL Server и как...   
11    8  февраля  2011                                     FAQ: SQL Azure   
12   24  октября  2012  Полное руководство: средства и способы миграци...   
13      14  июня  2016  Полный список флагов трассировки Microsoft SQL...   
14          25  июня    Технология SQL-файл, препроцессор для T-SQL, “...   
15   16  августа  2012  Масштабирование и особенности разработки для S...   
16  13  сентября  2012  Сравнивая и противопоставляя Windows Azure Tab...   
17    9  октября  2012                      Генератор SQL запросов на PHP   
18   13  декабря  2012  Всё что вы стеснялись спросить о бэкапах Micro...   
19    29  апреля  2014  SQL Server в облаке Microsoft Azure: PaaS vs IaaS   
20   25  декабря  2017  Новый подход к спортивному анализу данных: как...   
21  28  сентября  2018  Приглашаем на Sberbank Data Science Journey 20...   
22     9  ноября  2018  Прямая трансляция Sberbank Data Science Day 10...   
23   28  декабря  2016  Отчёт со Sberbank Data Science Day: решения, п...   
24     7  ноября  2017  Приглашаем на Sberbank Data Science Day 11 ноября   

                                                 link  \
0                  https://habr.com/ru/news/t/585346/   
1   https://habr.com/ru/company/yandex_praktikum/n...   
2   https://habr.com/ru/company/galssoftware/news/...   
3                  https://habr.com/ru/news/t/684376/   
4                    https://habr.com/ru/post/696274/   
5   https://habr.com/ru/company/dataline/news/t/53...   
6   https://habr.com/ru/company/netologyru/news/t/...   
7   https://habr.com/ru/company/rosbank/news/t/672...   
8                  https://habr.com/ru/news/t/675074/   
9                  https://habr.com/ru/news/t/581572/   
10                   https://habr.com/ru/post/281194/   
11                   https://habr.com/ru/post/113379/   
12  https://habr.com/ru/company/microsoft/blog/155...   
13                   https://habr.com/ru/post/303212/   
14                   https://habr.com/ru/post/672084/   
15  https://habr.com/ru/company/microsoft/blog/149...   
16                   https://habr.com/ru/post/151394/   
17                   https://habr.com/ru/post/154245/   
18                   https://habr.com/ru/post/162497/   
19  https://habr.com/ru/company/microsoft/blog/221...   
20                https://habr.com/ru/article/344806/   
21  https://habr.com/ru/company/sberbank/blog/424635/   
22  https://habr.com/ru/company/sberbank/blog/429320/   
23                https://habr.com/ru/article/318160/   
24  https://habr.com/ru/company/sberbank/blog/341814/   

                                                 text likes  
0   Этичные хакеры из Go Secure обнаружили ошибку ...   +25  
1   Яндекс Практикум разработал курс «SQL для рабо...    +2  
2                                                        +3  
3   Американский журнал IEEE Spectrum опубликовал ...    +3  
4   Статей о работе с PostgreSQL и её преимущества...   +76  
5   Новый кластер с Intel Optane, NVMe и процессор...    +5  
6   С 15 октября начнётся курс Нетологии «Введение...    +3  
7   Второй год подряд мы совместно с